In [1]:
import pandas as pd
import numpy as np

**- def developer(desarrollador : str) :**

    Cantidad de items y porcentaje de contenido Free por año según empresa desarrolladora.

In [2]:
def developer(desarrollador: str):
    #Levanto los datos
    df_steam_games = pd.read_parquet('..\\datasets\\3. Depurado y Reducido\\steam_games_dep_reducido.parquet', columns=['developer','year','free'])
    

    # Verifico si el desarrollador existe en el DataFrame
    if desarrollador.lower() not in df_steam_games['developer'].str.lower().values:
        return f"El desarrollador '{desarrollador}' no se encuentra en el dataset."

    # Filtro por desarrollador
    df_filtered = df_steam_games[df_steam_games['developer'].str.lower()  == desarrollador.lower()]

    # Obtenemos un DF del desarrollador con la cantidad de items por año
    total_items_per_year = df_filtered.groupby('year').size().reset_index(name='total_items')

    # Obtenemos un DF con la cantidad de items "Free" or año
    free_items_per_year = df_filtered[df_filtered['free'] == True].groupby('year').size().reset_index(name='free_items')

    # Left Join de los DF con "year" como clave.
    result = pd.merge(total_items_per_year, free_items_per_year, on='year', how='left').fillna(0)

    # Calculamos el porcentaje de contenido gratuito
    result['percentage_free'] = (result['free_items'] / result['total_items']) * 100

    # Le doy formato a las columnas del DF
    result['year'] = result['year'].astype(int)  # Asegúrate que sea entero
    result['total_items'] = result['total_items'].astype(int)  # Asegúrate que sea entero
    result['free_items'] = result['free_items'].astype(int)    # Asegúrate que sea entero
    result['percentage_free'] = result['percentage_free'].map('{:.2f}%'.format)  # Formato de porcentaje

    return result

In [3]:
# Llamada a la función
desarrollador = 'Valve'
resultados = developer(desarrollador)
resultados

,year,total_items,free_items,percentage_free
0,1998,1,0,0.00%
1,1999,1,0,0.00%
2,2000,2,0,0.00%
3,2001,1,0,0.00%
4,2003,1,0,0.00%
5,2004,5,0,0.00%
6,2005,1,0,0.00%
7,2006,2,0,0.00%
8,2007,3,1,33.33%
9,2008,1,0,0.00%


**- def userdata(user_id : str):**
    
    Debe devolver cantidad de dinero gastado por el usuario, el porcentaje de recomendación en base a reviews.recommend y cantidad de items.

In [4]:
def userdata(user_id: str):

    #Levanto los datos
    df_steam_games = pd.read_parquet('..\\datasets\\3. Depurado y Reducido\\steam_games_dep_reducido.parquet', columns=['item_id','price'])
    df_reviews = pd.read_parquet('..\\datasets\\3. Depurado y Reducido\\user_reviews_NLP.parquet', columns=['user_id','item_id','review','recommend'])
    df_user_items = pd.read_parquet('..\\datasets\\2. Depurado\\users_items_depurado.parquet', columns=['user_id','item_id'])

    

    # Me fijo que exista user_id en df_user_items
    if user_id not in df_user_items['user_id'].values:
        return print(f"El user_id {user_id} no se encuentra en user_items.")

    # Filtrar por el user_id en los dataframe
    user_items = df_user_items[df_user_items['user_id'] == user_id]
    user_items = user_items[user_items['item_id'].notnull()]  # Elimino nulos por si quedaron

    # Me fijo que haya items_id en user_items con el user_id
    if user_items.empty:
        print(f"No se encontraron items para el user_id {user_id} en user_items.")
    
    else:
        
        # Obtenemos los items del DF steam_games a partir de los items que tenga el usuario en user:
        items = df_steam_games[df_steam_games['item_id'].isin(user_items['item_id'])]

        if items.empty:
            return print(f"No se encontraron items en steam_games para el user_id {user_id}.")
        else:
            
            #Filtro las reviews del user_id
            user_reviews = df_reviews[(df_reviews['user_id'] == user_id) & (df_reviews['review'].notnull())]
            
            # Obtenemos los items del DF steam_games a partir de los items que tenga el usuario en user:
            items = df_steam_games[df_steam_games['item_id'].isin(user_items['item_id'])]
                        
            # Calcular dinero gastado
            total_spent = items['price'].sum()

            # Calcular el porcentaje de recomendación
            if not user_reviews.empty:
                total_reviews = user_reviews['review'].count() # Total de reviews
                recommended_reviews = user_reviews[user_reviews['recommend'] == True]['recommend'].count() # Total de recomendados
                percentage_recommendation = (recommended_reviews / total_reviews) * 100 if total_reviews > 0 else 0 # Porcentaje de recomendados de las reviews. 
            else:
                percentage_recommendation = 0

            # Contar cantidad de items
            item_count = user_items.shape[0]

            # Formatear el resultado
            result = {
                "Usuario": user_id,
                "Dinero gastado": f"{total_spent} USD",
                "% de recomendación": f"{percentage_recommendation:.2f}%",
                "cantidad de items": item_count
            }

            return result

In [5]:
# Llamada a la función
user_id = 'STEAM0082987612'
resultados = userdata(user_id)
print(resultados)

{'Usuario': 'STEAM0082987612', 'Dinero gastado': '1209.26 USD', '% de recomendación': '85.71%', 'cantidad de items': 106}


**-def UserForGenre( genero : str ):** 

    Debe devolver el usuario que acumula más horas jugadas para el género dado y una lista de la acumulación de horas jugadas por año de lanzamiento.

In [13]:
def UserForGenre(genero: str):
    # Levantar los datos
    df_steam_games = pd.read_parquet('..\\datasets\\3. Depurado y Reducido\\steam_games_dep_reducido.parquet', columns=['genres','item_id','year'])
    df_user_items = pd.read_parquet('..\\datasets\\2. Depurado\\users_items_depurado.parquet', columns=['user_id','item_id','playtime_forever']) 


    # Filtro los juegos por género - genre es un array dentro del DF.
    juegos_genero = df_steam_games[df_steam_games['genres'].apply(lambda x: genero in x if x is not None else False)]

    # Verifico que el género se encuentre en genres
    if juegos_genero.empty:
        return f"No se encontraron juegos para el género '{genero}'."

    # Inner Join de los datos de juegos y tiempo jugado
    user_play_time = pd.merge(df_user_items, juegos_genero[['item_id', 'year']], on='item_id', how='inner')

    # Sumo las horas jugadas por usuario y año
    horas_jugadas = user_play_time.groupby(['user_id', 'year'])['playtime_forever'].sum().reset_index()

    # Calculo el total de horas jugadas por cada usuario para el género
    total_horas_por_usuario = horas_jugadas.groupby('user_id')['playtime_forever'].sum().reset_index()

    # Encontrar el usuario con más horas jugadas
    mejor_usuario = total_horas_por_usuario.loc[total_horas_por_usuario['playtime_forever'].idxmax()]

    # Filtrar las horas jugadas por año para el mejor usuario
    horas_por_año = horas_jugadas[horas_jugadas['user_id'] == mejor_usuario['user_id']]

    # Formatear el resultado
    resultado = {
        "Usuario con más horas jugadas para Género {}".format(genero): mejor_usuario['user_id'],
        "Horas jugadas": [{"Año": int(year), "Horas": int(hours)} for year, hours in zip(horas_por_año['year'], horas_por_año['playtime_forever'])]
    }

    return resultado

In [14]:
# Ejemplo de uso
resultado = UserForGenre('Racing')
print(resultado)

{'Usuario con más horas jugadas para Género Racing': 'DownSyndromeKid', 'Horas jugadas': [{'Año': 1994, 'Horas': 0}, {'Año': 1995, 'Horas': 318}, {'Año': 1996, 'Horas': 0}, {'Año': 1999, 'Horas': 0}, {'Año': 2005, 'Horas': 91}, {'Año': 2006, 'Horas': 74553}, {'Año': 2007, 'Horas': 147814}, {'Año': 2008, 'Horas': 147707}, {'Año': 2009, 'Horas': 0}, {'Año': 2010, 'Horas': 0}, {'Año': 2011, 'Horas': 295361}, {'Año': 2013, 'Horas': 74271}, {'Año': 2014, 'Horas': 990}, {'Año': 2015, 'Horas': 2155}, {'Año': 2016, 'Horas': 185}]}


**- def best_developer_year( año : int ):** 

    Devuelve el top 3 de desarrolladores con juegos MÁS recomendados por usuarios para el año dado. (reviews.recommend = True y comentarios positivos)

In [10]:
def best_developer_year(año: int):
    
    #Levanto los datos
    df_steam_games = pd.read_parquet('..\\datasets\\3. Depurado y Reducido\\steam_games_dep_reducido.parquet', columns=['item_id','year','developer'])
    df_reviews = pd.read_parquet('..\\datasets\\3. Depurado y Reducido\\user_reviews_NLP.parquet', columns=['user_id','item_id','recommend','sentiment_value'])
    
    # Filtrar los juegos por año
    juegos_año = df_steam_games[df_steam_games['year'] == año]

    # Verifico si hay juegos en el año dado
    if juegos_año.empty:
        return f"No se encontraron juegos para el año {año}."

    # Filtrar las reviews recomendación "True" y reseñas "Positivas" del NLP
    reseñas_positivas = df_reviews[(df_reviews['recommend'] == True) & (df_reviews['sentiment_value'] == '2')]

    # Verificar si hay reseñas positivas
    if reseñas_positivas.empty:
        return f"No se encontraron reseñas positivas para el año {año}."

    # Inner join de los DF con la columna item_id como id
    juegos_reseñas = pd.merge(juegos_año, reseñas_positivas, on='item_id', how='inner')

    # Verificar si hay reseñas después del merge
    if juegos_reseñas.empty:
        return f"No hay reseñas positivas para los juegos del año {año}."

    # Cuento la cantidad de recomendaciones positivas por desarrollador
    recomendados_por_desarrollador = juegos_reseñas.groupby('developer').size().reset_index(name='count')

    # Obtengo el top 3 desarrolladores (sin ordenar)
    top_desarrolladores = recomendados_por_desarrollador.nlargest(3, 'count')

    # Formatear el resultado en el formato deseado
    resultado = []
    for idx, row in enumerate(top_desarrolladores.itertuples(), start=1):
        puesto = f"Puesto {idx}"
        resultado.append({puesto: row.developer})

    return resultado

In [11]:
# Llamada a la función
año = 2001  # Cambia este valor según el año que quieras consultar
resultados = best_developer_year(año)
print(resultados)

[{'Puesto 1': 'Firaxis Games'}, {'Puesto 2': 'Remedy Entertainment'}, {'Puesto 3': 'Gearbox Software'}]


**- def developer_reviews_analysis( desarrolladora : str ):** 
    
    Según el desarrollador, se devuelve un diccionario con el nombre del desarrollador como llave y una lista con la cantidad total de registros de reseñas de usuarios que se encuentren categorizados con un análisis de sentimiento como valor positivo o negativo.

In [16]:
def developer_reviews_analysis(desarrolladora: str):

    #Levanto los datos
    df_steam_games = pd.read_parquet('..\\datasets\\3. Depurado y Reducido\\steam_games_dep_reducido.parquet',columns=['item_id','developer'])
    df_reviews = pd.read_parquet('..\\datasets\\3. Depurado y Reducido\\user_reviews_NLP.parquet',columns=['item_id','sentiment_value'])
    
    
    # Filtrar los juegos del desarrollador especificado. Por las dudas lo paso a minúsculas
    juegos_desarrolladora = df_steam_games[df_steam_games['developer'].str.lower() == desarrolladora.lower()]
    
    # Verifico que haya juegos para este desarrollador y sino devuelvo 0
    if juegos_desarrolladora.empty:
        return {desarrolladora: {'Negative': 0, 'Positive': 0}}

    # Obtener los item_ids de los juegos de la desarrolladora
    item_ids = juegos_desarrolladora['item_id']

    # Filtrar las reseñas relacionadas con esos juegos
    reseñas_desarrolladora = df_reviews[df_reviews['item_id'].isin(item_ids)]

    # Contar las reseñas clasificadas como positivas y negativas resultantes del NLP
    total_positivas = reseñas_desarrolladora[reseñas_desarrolladora['sentiment_value'] == '2'].shape[0]
    total_negativas = reseñas_desarrolladora[reseñas_desarrolladora['sentiment_value'] == '0'].shape[0]

    # Crear el diccionario de resultados
    resultado = {
        desarrolladora: {
            'Negative': int(total_negativas),
            'Positive': int(total_positivas)
        }
    }

    return resultado



In [17]:
# Llamada a la función
desarrolladora = 'Bohemia Interactive'  # Cambia esto según el desarrollador que desees analizar
resultados = developer_reviews_analysis(desarrolladora)
print(resultados)

{'Bohemia Interactive': {'Negative': 323, 'Positive': 568}}
